In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from sklearn.linear_model import LinearRegression
import numpy as np

# Load the data
file_path = 'Overall Fleet Report_2025-01-01_-_2025-04-10.csv'
df = pd.read_csv(file_path)

# Convert and sort date
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Function to apply filters and plot with regression
def plot_metric_with_trend(group_by, metric, time_range):
    # Filter by time range
    max_date = df['Date'].max()
    if time_range == 'Last week':
        min_date = max_date - pd.Timedelta(weeks=1)
    elif time_range == 'Last month':
        min_date = max_date - pd.DateOffset(months=1)
    elif time_range == 'Yesterday':
        min_date = max_date.normalize() - pd.Timedelta(days=1)
        max_date = min_date + pd.Timedelta(days=1)
    else:  # All data
        min_date = df['Date'].min()

    df_filtered = df[(df['Date'] >= min_date) & (df['Date'] <= max_date)]

    # Resampling
    if group_by == 'Day':
        grouped = df_filtered.resample('D', on='Date').mean()
    elif group_by == 'Week':
        grouped = df_filtered.resample('W-MON', on='Date').mean()
    elif group_by == '15 Days':
        grouped = df_filtered.resample('15D', on='Date').mean()
    elif group_by == 'Month':
        grouped = df_filtered.resample('M', on='Date').mean()
    else:
        return

    grouped = grouped[[metric]].dropna()

    if len(grouped) < 2:
        print("Not enough data to analyze. Try a different time range or grouping.")
        return

    # Linear regression
    X = np.arange(len(grouped)).reshape(-1, 1)
    y = grouped[metric].values
    model = LinearRegression()
    model.fit(X, y)
    trend = model.predict(X)
    percent_change = (trend[-1] - trend[0]) / np.mean(y) * 100

    # Plot
    plt.figure(figsize=(14, 6))
    plt.plot(grouped.index, y, marker='o', label=metric)
    plt.plot(grouped.index, trend, linestyle='--', color='red', label='Trend Line')
    plt.title(f'{metric} ({time_range}) Grouped by {group_by}\nChange: {percent_change:.2f}% of average')
    plt.xlabel('Date')
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Full interactive widget
interact(
    plot_metric_with_trend,
    group_by=['Day', 'Week', '15 Days', 'Month'],
    metric=[
        'Operational Intensity (%)',
        'Active Time (%)',
        'Capacity Utilization (%)',
        'Active Equipment (#)'
    ],
    time_range=[
        'All data',
        'Last week',
        'Last month',
        'Yesterday'
    ]
)


interactive(children=(Dropdown(description='group_by', options=('Day', 'Week', '15 Days', 'Month'), value='Day…

<function __main__.plot_metric_with_trend(group_by, metric, time_range)>

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from datetime import timedelta

# Load the data
file_path = 'Overall Fleet Report_2025-01-01_-_2025-04-10.csv'
df = pd.read_csv(file_path)

# Convert 'Date' to datetime and localize to UTC if needed
df['Date'] = pd.to_datetime(df['Date'])
if df['Date'].dt.tz is None:
    df['Date'] = df['Date'].dt.tz_localize('UTC')

# Convert to GMT-6
df['Date'] = df['Date'].dt.tz_convert('Etc/GMT+6')

# Define metrics
metrics = [
    'Operational Intensity (%)',
    'Active Time (%)',
    'Capacity Utilization (%)',
    'Active Equipment (#)'
]

# Interactive function
def show_hourly_summary(time_range):
    max_date = df['Date'].max()

    if time_range == 'Last week':
        min_date = max_date - timedelta(weeks=1)
    elif time_range == 'Last 15 days':
        min_date = max_date - timedelta(days=15)
    elif time_range == 'Last month':
        min_date = max_date - pd.DateOffset(months=1)
    elif time_range == 'Today':
        min_date = max_date.normalize()
    else:  # All data
        min_date = df['Date'].min()

    df_filtered = df[(df['Date'] >= min_date) & (df['Date'] <= max_date)].copy()
    df_filtered['Hour'] = df_filtered['Date'].dt.hour
    hourly_means_local = df_filtered.groupby('Hour')[metrics].mean()

    summary_local = []

    for metric in metrics:
        best_hour = hourly_means_local[metric].idxmax()
        worst_hour = hourly_means_local[metric].idxmin()
        best_value = hourly_means_local[metric].max()
        worst_value = hourly_means_local[metric].min()

        summary_local.append({
            'Metric': metric,
            'Best Hour (GMT-6)': best_hour,
            'Best Value': round(best_value, 2),
            'Worst Hour (GMT-6)': worst_hour,
            'Worst Value': round(worst_value, 2)
        })

        # Plot
        plt.figure(figsize=(12, 5))
        plt.plot(hourly_means_local.index, hourly_means_local[metric], marker='o', linewidth=2)
        plt.axvline(x=best_hour, color='green', linestyle='--', label=f'Best Hour: {best_hour}h')
        plt.axvline(x=worst_hour, color='red', linestyle='--', label=f'Worst Hour: {worst_hour}h')
        plt.title(f'{metric} by Hour of Day (GMT-6) - {time_range}')
        plt.xlabel('Hour of Day')
        plt.ylabel(metric)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

    summary_df = pd.DataFrame(summary_local)
    display(summary_df)

# Widget interface
interact(
    show_hourly_summary,
    time_range=[
        'All data',
        'Last week',
        'Last 15 days',
        'Last month',
        'Today'
    ]
)


interactive(children=(Dropdown(description='time_range', options=('All data', 'Last week', 'Last 15 days', 'La…

<function __main__.show_hourly_summary(time_range)>